In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [4]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [8]:
columns = [
    "id", "title", "budget", "genre", "popularity", "release_date", "revenue",
    "runtime", "vote_average"
]

target = ["revenue"]

In [12]:
# Load the data
file_path = Path('movie_data.csv')
df = pd.read_csv(file_path, skiprows=0)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
#issued_mask = df['revenue'] != 'Issued'
#df = df.loc[issued_mask]

# convert interest rate to numerical
#df['int_rate'] = df['int_rate'].str.replace('%', '')
#df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
#x = {'Current': 'low_risk'}   
#df = df.replace(x)

#x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
#df = df.replace(x)

#df.reset_index(inplace=True, drop=True)

df.head()

,id,title,budget,genre,popularity,release_date,revenue,runtime,vote_average
0,100,"Lock, Stock and Two Smoking Barrels",1350000,Crime,4.60786,1998-03-05,3897569.0,105.0,7.5
1,100,"Lock, Stock and Two Smoking Barrels",1350000,Comedy,4.60786,1998-03-05,3897569.0,105.0,7.5
2,10003,The Saint,68000000,Action,10.97633,1997-04-03,118063304.0,116.0,5.9
3,10003,The Saint,68000000,Adventure,10.97633,1997-04-03,118063304.0,116.0,5.9
4,10003,The Saint,68000000,Science Fiction,10.97633,1997-04-03,118063304.0,116.0,5.9
